In [1]:
import numpy as np
import pandas as pd

%store -r columnNames

#pd.set_option('display.max_columns', None)  # or 1000
# pd.set_option('display.max_rows', None)  # or 1000
# pd.set_option('display.max_colwidth', None)  # or 199

# pd.describe_option('display')

# Analyze Data

In [2]:
surveyData=pd.read_csv('justCompletedAttempts.csv', sep=';')

print("Anzahl an kompletten Daten:", len(surveyData))

Anzahl an kompletten Daten: 203
